# Preprocessing the Data

The rawest data form was very much unstructured and unusable. To ease debugging and data structure, the tables are first converted into a more manageable form, i.e. IDs are standardized, grades are converted into numerical scores, different concentration into different files, and so on. This is done with comparison with other data sources, such as the past curricula, correct credit scores, and course name changes. With less than a hundred student records, the initial data cleansing was better done with a spreadsheet program.

The following `csv` files are already processed data, ready to be manipulated futher using python libraries.

## Loading the data

There are three `csv` files to be read:
* `course.csv` contains the list of all courses, including the credits and curriculum
* `main.csv` contains score logs of each student
* `student.csv` contains student data, including their cohort and major

We first declare the file names for ease of use

In [1]:
import os
import time

import pandas as pd
import numpy as np

data_path = os.path.join('../data/TIF/')
courses_file = 'course.csv'
scores_file = 'main.csv'
students_file = 'student.csv'

To easily manipulate data as tables, we will use `pandas` dataframes. The previously declared `csv` files are then loaded into dataframes. We will store these dataframes into three separate variables, each for a corresponding `csv` file.

In [2]:
df_courses = pd.read_csv(
    os.path.join(data_path, courses_file),
    usecols=['CourseCode', 'CourseName', 'Credit', 'T2016', 'T2015', 'T2014', 'T2013', 'T2012'],
    dtype={'CourseCode': 'str', 'CourseName': 'str', 'Credit': 'int32', 'T2016': 'int32', 'T2015': 'int32', 'T2014': 'int32', 'T2013': 'int32', 'T2012': 'int32'})

df_scores = pd.read_csv(
    os.path.join(data_path, scores_file),
    usecols=['TermCode', 'ID', 'CourseCode', 'GradePts', 'Term'],
    dtype={'TermCode': 'int32', 'ID': 'str', 'CourseCode': 'str', 'GradePts': 'float32', 'Term': 'str'})

df_students = pd.read_csv(
    os.path.join(data_path, students_file),
    usecols=['ID', 'Cohort', 'Major'],
    dtype={'ID': 'str', 'Cohort': 'int32', 'Major': 'str'})

The `csv` files should now be stored in `df_courses`, `df_scores`, and `df_students` respectively. Let's check if the data is correcly loaded by looking into the first few rows of the dataframe.

In [3]:
df_courses.head() # Checking course dataframe

,CourseCode,CourseName,Credit,T2016,T2015,T2014,T2013,T2012
0,INF0011,ALGORITMA & PEMROGRAMAN,4,8,8,8,8,0
1,INF0021,ALJABAR LINIER & MATRIKS,2,8,8,0,0,0
2,INF0022,ALJABAR LINIER & MATRIKS,3,0,0,8,8,8
3,INF0031,ANALISIS ALGORITMA,2,8,8,8,8,0
4,INF0041,ARSITEKTUR PIRANTI LUNAK,3,2,2,0,0,0


In [4]:
df_scores.head() # Checking score dataframe

,Term,ID,CourseCode,GradePts,TermCode
0,2012-1,S001,INF0132,1.0,20121
1,2012-1,S001,INF0221,2.3,20121
2,2012-1,S001,INF0111,2.0,20121
3,2012-1,S001,INF0371,3.3,20121
4,2012-1,S002,INF0132,1.0,20121


In [5]:
df_students.head() # Checking student dataframe

,ID,Cohort,Major
0,S001,2012,MI
1,S002,2012,SE
2,S003,2012,MI
3,S004,2012,MI
4,S005,2012,MI


## Let's explore

Seems that the dataframes are correctly loaded. Let's explore the data a bit more.

Since we are going to use `df_scores` for the actual training, we should compare it to the other tables. We are gonig to compare two things:

* The number of unique students in the `df_scores` table against the number of unique students in the `df_students` table.
* The number of unique courses (not including FLA), in the `df_scores` table against the number of available courses in the `df_courses` table.

In [6]:
n_students = len(df_scores.ID.unique())
n_students_ch = len(df_students.ID.unique())
n_courses = len(df_scores.CourseCode.unique()) # minus course FLA
n_courses_ch = len(df_courses.CourseCode.unique())

print('Unique students: {} / {}'.format(n_students, n_students_ch))
print('Unique courses: {} / {}'.format(n_courses, n_courses_ch))

Unique students: 89 / 89
Unique courses: 75 / 81


We can see that not all courses have scores in the score log&mdash;only 75 out of the 81 courses exist in the score log.

We'll then check the distribution of scores across courses, just for the sake of it.

In [7]:
# Checking scores' distribution

df_scores_cnt_gr = pd.DataFrame(df_scores.groupby('GradePts').size(), columns=['count'])
df_scores_cnt_gr

,count
GradePts,
0.0,24
1.0,25
1.7,197
2.0,298
2.3,397
2.7,478
3.0,525
3.3,483
3.7,362


## Cleaning up the dirt

Exploring the `csv` data further shows that several student records have malformed data. We'll remove these records, since it will mess up the training.

A total of 11 student records will be removed:
* The first 8 are 2012 cohort students, which have a different curriculum, enabling them to take courses optionally.
* The last three ahve too much missing courses and will be removed.

In [8]:
# Removing S001-S008, S047, S048, S089

unused_students = ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S047', 'S048', 'S089']

df_scores = df_scores[~df_scores.ID.isin(unused_students)]
df_students = df_students[~df_students.ID.isin(unused_students)]

Thesis courses won't be included as part of the model training. This is due to the fact that these don't necessarily reflect a student's concentration&mdash;an "Interactive Media Design & Development" concentration student might take a "Software Engineering" topic for their thesis.

In [9]:
# Removing TA courses

thesis_course_codes = ["INF0671", "INF0681", "INF0691"]

df_scores = df_scores[~df_scores.CourseCode.isin(thesis_course_codes)]

The same course might have different `CourseCode`s. For example, the "Linear Algebra" course have codes `INF0021` and `INF0022`. The former only has 2 credits, while the latter has 3 credits. Because of this, a new column called `AdjCourse` is made to accomodate this issue.

In [10]:
# Adjusting similar courses

def adj_course(row):
    if row['CourseCode'] == 'INF0022':
        return 'INF0021'
    if row['CourseCode'] == 'INF0052':
        return 'INF0051'
    if row['CourseCode'] == 'INF0132':
        return 'INF0131'
    if row['CourseCode'] == 'INF0142':
        return 'INF0141'
    if row['CourseCode'] == 'INF0152':
        return 'INF0151'
    if row['CourseCode'] == 'INF0212':
        return 'INF0211'
    if row['CourseCode'] == 'INF0741':
        return 'INF0501'
    if row['CourseCode'] == 'INF0611':
        return 'INF0601'
    if row['CourseCode'] == 'INF0491':
        return 'INF0391'
    if row['CourseCode'] == 'INF0701':
        return 'INF0261'
    if row['CourseCode'] == 'INF0721' or row['CourseCode'] == 'INF0421':
        return 'INF0331'
    if row['CourseCode'] == 'INF0321' or row['CourseCode'] == 'INF0731':
        return 'INF0251'
    if row['CourseCode'] == 'INF0751':
        return 'INF0541'
    return row['CourseCode']

df_scores['AdjCourse'] = df_scores.apply(adj_course, axis=1)

## Splitting up concentrations

Data from different concentrations are still merged into one. Since we are going to build the model for each concentration, we need to separate the data from one and another.

**IMPORTANT:** These abbreviations will be used throughout the code

* `IMDD`: Interactive Media Design & Development
* `SE`: Software Engineering
* `MI`: Medical Informatics

In [11]:
# Splitting students based on concentration

student_imdd = df_students.groupby('Major').get_group('IMDD')
student_se = df_students.groupby('Major').get_group('SE')
student_mi = df_students.groupby('Major').get_group('MI')

In [12]:
print(
    'Student count:\nIMDD: {}\n{}\n\nSE: {}\n{}\n\nMI: {}\n{}\n\nTotal: {}'
    .format(
        student_imdd['ID'].count(),
        student_imdd.groupby('Cohort')['ID'].count(),
        student_se['ID'].count(),
        student_se.groupby('Cohort')['ID'].count(),
        student_mi['ID'].count(),
        student_mi.groupby('Cohort')['ID'].count(),
        df_students['ID'].count()
    )
)

Student count:
IMDD: 28
Cohort
2013     8
2014    13
2015     7
Name: ID, dtype: int64

SE: 23
Cohort
2015    23
Name: ID, dtype: int64

MI: 27
Cohort
2013    15
2014     2
2015    10
Name: ID, dtype: int64

Total: 78


In [13]:
# Splitting scores based on concentration

score_imdd = df_scores.loc[df_scores['ID'].isin(student_imdd['ID'])]
score_se = df_scores.loc[df_scores['ID'].isin(student_se['ID'])]
score_mi = df_scores.loc[df_scores['ID'].isin(student_mi['ID'])]

In [14]:
print(
    'Data count:\nIMDD: {} rows\n{}\n\nSE: {} rows\n{}\n\nMI: {} rows\n{}'
    .format(
        score_imdd['ID'].count(), 
        score_imdd.nunique(), 
        score_se['ID'].count(), 
        score_se.nunique(), 
        score_mi['ID'].count(), 
        score_mi.nunique()
    )
)

Data count:
IMDD: 985 rows
Term          16
ID            28
CourseCode    44
GradePts      10
TermCode      16
AdjCourse     38
dtype: int64

SE: 789 rows
Term          10
ID            23
CourseCode    35
GradePts      10
TermCode      10
AdjCourse     34
dtype: int64

MI: 938 rows
Term          16
ID            27
CourseCode    44
GradePts      10
TermCode      16
AdjCourse     38
dtype: int64


In [15]:
course_general=[]
course_imdd=[]
course_se=[]
course_mi=[]
exc_course=['INF0022', 'INF0052', 'INF0132', 'INF0142', 'INF0152', 'INF0212', 'INF0741', 'INF0611', 'INF0491', 'INF0701',\
            'INF0721', 'INF0321', 'INF0731', 'INF0751', 'INF0671', 'INF0681', 'INF0691']

for row in df_courses.itertuples(index=False):
    if (row.T2016 & 8 == 8 or row.T2015 & 8 == 8 or row.T2014 & 8 == 8 or row.T2013 & 8 == 8) and row.CourseCode not in exc_course:
        course_general.append(row.CourseCode)
    elif (row.T2016 & 4 == 4 or row.T2015 & 4 == 4 or row.T2014 & 4 == 4 or row.T2013 & 4 == 4):
        course_imdd.append(row.CourseCode)
    elif (row.T2016 & 2 == 2 or row.T2015 & 2 == 2 or row.T2014 & 2 == 2 or row.T2013 & 2 == 2):
        course_se.append(row.CourseCode)
    elif (row.T2016 & 1 == 1 or row.T2015 & 1 == 1 or row.T2014 & 1 == 1 or row.T2013 & 1 == 1):
        course_mi.append(row.CourseCode)

## Into the Matrix

If you are observant, you might find that the `scores` data are still stored as logs. This structure is not suitable for training. Hence, the scores will be transformed into a matrix form, where rows represent students and columns represent the scores for each course.

In [16]:
# Creating score matrix

imdd_temp = imdd = pd.DataFrame(index=student_imdd['ID'])
se_temp   = se   = pd.DataFrame(index=student_se['ID'])
mi_temp   = mi   = pd.DataFrame(index=student_mi['ID'])

# Creating columns

for r in course_general:
    imdd[r] = np.nan
    se[r] = np.nan
    mi[r] = np.nan

for r in course_imdd:
    imdd[r] = np.nan
    imdd_temp[r] = np.nan
    
for r in course_se:
    se[r] = np.nan
    se_temp[r] = np.nan
    
for r in course_mi:
    mi[r] = np.nan
    mi_temp[r] = np.nan

In [17]:
# Inputting data

for row in score_imdd.itertuples(index=False):
    imdd.loc[row.ID].at[row.AdjCourse] = row.GradePts
    imdd_temp.loc[row.ID].at[row.CourseCode] = row.GradePts
    
for row in score_se.itertuples(index=False):
    se.loc[row.ID].at[row.AdjCourse] = row.GradePts
    se_temp.loc[row.ID].at[row.CourseCode] = row.GradePts

for row in score_mi.itertuples(index=False):
    mi.loc[row.ID].at[row.AdjCourse] = row.GradePts
    mi_temp.loc[row.ID].at[row.CourseCode] = row.GradePts
    
    
# Removing na
imdd = imdd.dropna(axis=1, how='all')
se = se.dropna(axis=1, how='all')
mi = mi.dropna(axis=1, how='all')
imdd.round(2)
se.round(2)
mi.round(2)

imdd_temp = imdd.dropna(axis=1, how='all')
se_temp = se.dropna(axis=1, how='all')
mi_temp = mi.dropna(axis=1, how='all')
imdd_temp.round(2)
se_temp.round(2)
mi_temp.round(2)

,INF0011,INF0021,INF0031,INF0081,INF0121,INF0131,INF0141,INF0151,INF0161,INF0171,...,INF0631,INF0651,INF0051,INF0052,INF0201,INF0241,INF0351,INF0451,INF0511,INF0541
ID,,,,,,,,,,,,,,,,,,,,,
S012,2.3,2.7,2.3,1.7,3.7,2.7,2.0,2.7,3.3,2.3,...,NaN,3.0,3.7,NaN,3.0,1.7,3.3,NaN,3.3,3.7
S014,1.7,1.7,1.7,2.0,4.0,2.0,2.3,2.3,3.0,2.0,...,NaN,2.7,2.3,NaN,3.0,2.0,3.3,NaN,3.0,3.3
S015,2.3,4.0,2.0,2.0,4.0,1.7,2.0,3.3,2.7,2.0,...,NaN,3.3,0.0,0.0,3.3,2.3,3.0,NaN,2.0,2.3
S016,1.7,2.3,2.0,1.7,3.7,2.7,2.0,2.7,2.3,1.7,...,NaN,2.0,2.3,NaN,2.7,1.7,3.0,NaN,1.7,3.7
S018,1.7,1.7,1.7,1.7,2.0,3.0,1.7,3.0,2.7,2.0,...,NaN,2.3,2.0,NaN,3.0,3.0,2.3,NaN,2.3,2.0
S020,2.0,3.7,2.0,1.7,3.3,2.3,2.7,3.3,3.0,2.0,...,NaN,3.0,4.0,4.0,3.0,3.3,4.0,NaN,2.3,3.0
S022,2.0,3.7,2.3,2.0,3.7,3.0,2.3,3.3,3.0,2.0,...,NaN,3.0,4.0,4.0,3.0,3.7,2.7,NaN,3.0,3.0
S024,2.0,3.0,2.3,1.7,3.3,2.0,2.3,3.0,3.0,2.0,...,NaN,3.0,3.3,3.3,3.3,1.7,3.0,NaN,2.7,2.0
S025,1.7,3.0,2.0,1.7,3.3,1.7,2.3,2.7,2.3,2.0,...,NaN,2.7,2.3,NaN,2.7,2.7,3.7,NaN,2.3,2.7


## Credit lookup

This part is pretty self-explanatory as it loads credit data for each course code into a dataframe. It will later be used for calculating the concentration GPA.

In [18]:
# Creating credits' dataframe

credit_imdd = pd.DataFrame(course_imdd, columns=["Course"])
credit_se   = pd.DataFrame(course_se, columns=["Course"])
credit_mi   = pd.DataFrame(course_mi, columns=["Course"])

for row in credit_imdd.itertuples():
    credit_imdd['Credit'] = df_courses['Credit']

for row in credit_se.itertuples():
    credit_se['Credit'] = df_courses['Credit']

for row in credit_mi.itertuples():
    credit_mi['Credit'] = df_courses['Credit']
    
credit_imdd = credit_imdd.set_index('Course')
credit_se   = credit_se.set_index('Course')
credit_mi   = credit_mi.set_index('Course')

## Concentration GPA Calculation

We now already have the score matrices `imdd`, `se`, and `mi`. We also have the credit dataframe for easy credit lookup. Now we merge the two to derive the concentration GPA. **This is not the normal GPA**. The concentration GPA is derived from only using courses in a specific concentration. In other words, general courses won't be put into account for calculation.

\begin{equation*}
GPA = \frac{\sum_{i \in courses} credit_i \cdot score_i}{\sum_{i \in courses} credit_i}
\end{equation*}

In [19]:
# Calculating GPA based on concentration courses

imdd_temp['Score']  = 0
imdd_temp['Credit'] = 0

for row in score_imdd.itertuples(index=False):
    if row.CourseCode in course_imdd:
        imdd_temp.at[row.ID, 'Score']  += row.GradePts * credit_imdd.loc[row.CourseCode].at['Credit']
        imdd_temp.at[row.ID, 'Credit'] += credit_imdd.loc[row.CourseCode].at['Credit']
        
imdd_temp['GPA'] = imdd_temp['Score'] / imdd_temp['Credit']
imdd['GPA'] = imdd_temp['GPA']

se_temp['Score'] = 0
se_temp['Credit'] = 0

for row in score_se.itertuples(index=False):
    if row.CourseCode in course_se:
        se_temp.at[row.ID, 'Score'] += row.GradePts * credit_se.loc[row.CourseCode].at['Credit']
        se_temp.at[row.ID, 'Credit'] += credit_se.loc[row.CourseCode].at['Credit']
        
se_temp['GPA'] = se_temp['Score'] / se_temp['Credit']
se['GPA'] = se_temp['GPA']

mi_temp['Score'] = 0
mi_temp['Credit'] = 0

for row in score_mi.itertuples(index=False):
    if row.CourseCode in course_mi:
        mi_temp.at[row.ID, 'Score'] += row.GradePts * credit_mi.loc[row.CourseCode].at['Credit']
        mi_temp.at[row.ID, 'Credit'] += credit_mi.loc[row.CourseCode].at['Credit']
        
mi_temp['GPA'] = mi_temp['Score'] / mi_temp['Credit']
mi['GPA'] = mi_temp['GPA']

## Almost there!

Now we just need to store the processed data into a file.

In [20]:
# Output data

imdd.to_csv('../data/imdd.csv')
se.to_csv('../data/se.csv')
mi.to_csv('../data/mi.csv')